In [ ]:
import seaborn as sns
from scipy.ndimage.filters import gaussian_filter1d
from matplotlib.pyplot import figure
from datetime import datetime, timedelta

sns.set_theme()

In [ ]:
import numpy as np
import pylab


def thresholding_algo(y, lag, threshold, influence):
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.mean(y[0:lag])
    stdFilter[lag - 1] = np.std(y[0:lag])
    for i in range(lag, len(y) - 1):
        if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter [i-1]:
            if y[i] > avgFilter[i-1]:
                signals[i] = 1
            else:
                signals[i] = -1

            filteredY[i] = influence * y[i] + (1 - influence) * filteredY[i-1]
            avgFilter[i] = np.mean(filteredY[(i-lag):i])
            stdFilter[i] = np.std(filteredY[(i-lag):i])
        else:
            signals[i] = 0
            filteredY[i] = y[i]
            avgFilter[i] = np.mean(filteredY[(i-lag):i])
            stdFilter[i] = np.std(filteredY[(i-lag):i])

    return dict(signals = np.asarray(signals),
                avgFilter = np.asarray(avgFilter),
                stdFilter = np.asarray(stdFilter))



# Data
# y = np.array([1,1,1.1,1,0.9,1,1,1.1,1,0.9,1,1.1,1,1,0.9,1,1,1.1,1,1,1,1,1.1,0.9,1,1.1,1,1,0.9,
#        1,1.1,1,1,1.1,1,0.8,0.9,1,1.2,0.9,1,1,1.1,1.2,1,1.5,1,3,2,5,3,2,1,1,1,0.9,1,1,3,
#        2.6,4,3,3.2,2,1,1,0.8,4,4,2,2.5,1,1,1])
y = np.random.uniform (-0.2, 0.2, 50)
y[40] = 2
y[43]=1.5
y[41]=0.8
y[48]=0.8


y = gaussian_filter1d(y, sigma=1)

start_date = datetime(2015,7,1)
end_date = datetime(2015,7,1) + timedelta(days=len(y+1))
x = np.arange(start_date, end_date, timedelta(days=1)).astype(datetime)


# Settings: lag = 30, threshold = 5, influence = 0
lag = 30
threshold = 5
influence = 0

# Run algo with settings from above
result = thresholding_algo(y, lag=lag, threshold=threshold, influence=influence)

figure(figsize=(8, 6), dpi=100)


# Plot result
pylab.subplot(211)
pylab.plot(np.arange(1, len(y)+1), y)

pylab.plot(np.arange(1, len(y)+1),
           result["avgFilter"], color="cyan", lw=2)

# pylab.plot(np.arange(1, len(y)+1),
#            result["avgFilter"] + threshold * result["stdFilter"], color="green", lw=2)

# pylab.plot(np.arange(1, len(y)+1),
#            result["avgFilter"] - threshold * result["stdFilter"], color="green", lw=2)

pylab.subplot(212)
pylab.step(np.arange(1, len(y)+1), result["signals"], color="red", lw=2)
pylab.ylim(-1.5, 1.5)
    

In [ ]:
import matplotlib.pyplot as plt


figure(figsize=(8, 3), dpi=100)

x = np.arange (len(y))

m, b = np.polyfit(x, y, 1)
plt.scatter (x, y)
plt.plot(x, m*x + b, color='green')
# plt.xlabel('points per game')
# plt.ylabel('# of wins')
plt.show()